In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.iolib.summary2 import summary_col

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# Забираем датасет, предобработанный для проведения регрессионного анализа
!wget https://nc.cloud.cpur.ru/s/zAaYy5EDMySndAC/download/declarations_for_regression.xlsx

--2021-12-01 07:53:02--  https://nc.cloud.cpur.ru/s/zAaYy5EDMySndAC/download/declarations_for_regression.xlsx
Resolving nc.cloud.cpur.ru (nc.cloud.cpur.ru)... 84.201.157.40
Connecting to nc.cloud.cpur.ru (nc.cloud.cpur.ru)|84.201.157.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 774872 (757K) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘declarations_for_regression.xlsx’

declarations_for_re 100%[===================>] 756.71K  1.33MB/s    in 0.6s    

2021-12-01 07:53:04 (1.33 MB/s) - ‘declarations_for_regression.xlsx’ saved [774872/774872]



In [3]:
servants = pd.read_excel('declarations_for_regression.xlsx')
len(servants)

17223

In [21]:
servants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17223 entries, 0 to 17222
Data columns (total 9 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Unnamed: 0                        17223 non-null  int64  
 1   name                              17223 non-null  object 
 2   position_standard                 17223 non-null  object 
 3   gender                            17223 non-null  object 
 4   married                           17223 non-null  object 
 5   year                              17223 non-null  int64  
 6   state_agency_short                17223 non-null  object 
 7   children                          17223 non-null  int64  
 8   income_diff_month_const_чиновник  17223 non-null  float64
dtypes: float64(1), int64(3), object(5)
memory usage: 1.2+ MB


In [4]:
# создаем переменную: наличине несоврешеннолетних детей
servants['if_children'] = np.where(servants['children'] == 0, 'no', 'yes')

In [5]:
#логарифмируем месячный доход чиновников
servants['log_monthly_income'] = np.log(servants['income_diff_month_const_чиновник'])

In [6]:
# создаем категориальные переменную года
servants['year_cat'] = servants['year'].astype(str)

## 1. Регрессия на данных всех министерств

In [31]:
list_predictors = ['gender', ' + married', ' + if_children', ' + state_agency_short',
                   ' + year_cat', ' + position_standard']

In [35]:
# Последовательно добавляем объясняющие переменные из списка
models_list = []
formula = 'log_monthly_income ~ '

for predictor in list_predictors:
  formula += predictor
  model = ols(formula, data = servants).fit()
  models_list.append(model)

Формируем общую таблицу с результатами регрессии на всех данных

In [36]:
res_table_all = summary_col(models_list, 
                            stars=True,float_format='%0.3f', info_dict={'N':lambda x: "{0:d}".format(int(x.nobs)),
                             'R2':lambda x: "{:.3f}".format(x.rsquared)},
                             model_names=['(1)','(2)','(3)', '(4)', '(5)', '(6)'])

**Печатаем результаты**

In [ ]:
res_table_all.tables[0]

## 2. Регрессия по группам министерств

In [14]:
# функция для последовательного добавления объясняющих переменных
# и формирования общей таблицы с результатами

def fit_regression_models(data):

  models = [
  #Только пол
  'log_monthly_income ~ gender',
  #Добавляем семейный статус и наличие детей
  'log_monthly_income ~ gender + married + if_children',
  #Добавляем министерство и год
  'log_monthly_income ~ gender + married + if_children + state_agency_short + year_cat',
  #Добавляем должность
  'log_monthly_income ~ gender + married + if_children + state_agency_short + year_cat + position_standard',
  #Добавляем эффект взаимодействия
  'log_monthly_income ~ gender + married + if_children + state_agency_short + year_cat + position_standard + gender*married  + gender*if_children'
  ]

  #проводим регрессию на всех вариантах моделей
  trained_models = list(map(lambda x: ols(x, data=data).fit() , models))

  #Формируем таблицу с результатами регрессии без взаимодействия
  res_table_no_int = summary_col(trained_models[0:4],
                          stars=True, float_format='%0.3f',
                           info_dict={'N':lambda x: "{0:d}".format(int(x.nobs)),
                                      'R2':lambda x: "{:.3f}".format(x.rsquared)},
                           model_names=['(1)','(2)','(3)', '(4)'])

  result = [res_table_no_int, trained_models[3], trained_models[4]]
  return result



Формируем группы согласно классификации, описаной в тексте аналитической записки



In [15]:
# Группа 1
state_list = ['Минэнерго', 'Минфин', 'Минтранс', 'Минприроды']
group_1 = servants[servants['state_agency_short'].isin(state_list)]
len(group_1)

4587

In [16]:
# Группа 2
state_list = ['Минвостокразвития', 'Минэкономразвития', 'Минспорт']
group_2 = servants[servants['state_agency_short'].isin(state_list)]
len(group_2)

3783

In [17]:
# Группа 3
state_list = ['Минцифры', 'Минобр', 'Минюст']
group_3 = servants[servants['state_agency_short'].isin(state_list)]
len(group_3)

2711

In [18]:
# Группа 4
state_list = ['Минкультуры', 'Минстрой', 'Минтруд', 'Минздрав', 'Минсельхоз', 'Минпромторг']
group_4 = servants[servants['state_agency_short'].isin(state_list)]
len(group_4)

6142

Создаем и тренируем модели для каждой группы

In [19]:
res_table_1, mdl_1_no_int_full, mdl_1_int_full = fit_regression_models(group_1)
res_table_2, mdl_2_no_int_full, mdl_2_int_full = fit_regression_models(group_2)
res_table_3, mdl_3_no_int_full, mdl_3_int_full = fit_regression_models(group_3)
res_table_4, mdl_4_no_int_full, mdl_4_int_full = fit_regression_models(group_4)

Формируем сводную таблицу результатов по всем группам без взаимодействия

In [20]:
res_table_groups_no_int = summary_col([mdl_1_no_int_full, mdl_2_no_int_full, mdl_3_no_int_full,
                                       mdl_4_no_int_full], stars=True, float_format='%0.3f',
                                       info_dict={'N':lambda x: "{0:d}".format(int(x.nobs)),
                             'R2':lambda x: "{:.3f}".format(x.rsquared)},
                              model_names=['Группа 1', 'Группа 2', 'Группа 3', 'Группа 4'])

Формируем сводную таблицу результатов по всем группам с эффектом взаимодействия

In [21]:
res_table_groups_int = summary_col([mdl_1_int_full, mdl_2_int_full, mdl_3_int_full,
                                    mdl_4_int_full], stars=True, float_format='%0.3f',
                                  info_dict={'N':lambda x: "{0:d}".format(int(x.nobs)),
                             'R2':lambda x: "{:.3f}".format(x.rsquared)},
                              model_names=['Группа 1', 'Группа 2', 'Группа 3', 'Группа 4'])

**Печатаем результаты регрессионного анализа**

In [ ]:
#Группа 1
res_table_1.tables[0]

In [ ]:
#Группа 2
res_table_2.tables[0]

In [ ]:
#Группа 3
res_table_3.tables[0]

In [ ]:
#Группа 4
res_table_4.tables[0]

In [ ]:
# Полные модели по всем группам без взаимодействия
res_table_groups_no_int.tables[0]

In [ ]:
# Полные модели по всем группам со взаимодействием
res_table_groups_int.tables[0]